In [19]:
# ================================
#  DATA PREPARATION PIPELINE (IMPROVED)
#  + Cuisine taxonomy enrichment (NO postal code / NO area)
# ================================

import pandas as pd
import numpy as np
import re
import unicodedata

# -------------------------
# 1) LOAD
# -------------------------
df = pd.read_csv("zurich.csv")

print("Shape before cleaning:", df.shape)
print("\nColumns:\n", df.columns)
print("\nMissing values before cleaning:")
print(df.isna().sum())

# -------------------------
# 2) CLEAN RATING (ONCE)
# -------------------------
df["rating"] = (
    df["rating"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.strip()
)
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")

# Keep only rows with rating (common for rating-based analysis)
df = df.dropna(subset=["rating"]).copy()

# -------------------------
# 3) CLEAN MINIMUM (ONCE)
# -------------------------
df["minimum_clean"] = (
    df["minimum"]
    .astype(str)
    .str.replace("Min.", "", regex=False)
    .str.replace("CHF", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.strip()
)
df["minimum_clean"] = pd.to_numeric(df["minimum_clean"], errors="coerce").fillna(0)

# -------------------------
# 4) REMOVE DUPLICATES (ONCE)
# -------------------------
df = df.drop_duplicates(subset=["name", "web_scraper_start_url"]).copy()

Shape before cleaning: (201, 6)

Columns:
 Index(['web_scraper_order', 'web_scraper_start_url', 'name', 'rating', 'description', 'minimum'], dtype='object')

Missing values before cleaning:
web_scraper_order        0
web_scraper_start_url    0
name                     0
rating                   1
description              0
minimum                  0
dtype: int64


In [ ]:
# ==========================================================
# 5) CUISINE TAXONOMY ENRICHMENT (IMPROVED)
# ==========================================================

def normalize_text(s: str) -> str:
    """Lowercase, normalize unicode, unify separators, remove extra spaces."""
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKC", s)
    # turn slashes and ampersands into spaces for consistent matching
    s = s.replace("/", " ").replace("&", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def split_tags(description: str) -> list:
    """Split comma-separated tags, normalize, de-duplicate (order preserved)."""
    desc = normalize_text(description)
    if desc == "":
        return []
    parts = [normalize_text(p) for p in desc.split(",")]
    parts = [p for p in parts if p]

    seen, out = set(), []
    for p in parts:
        if p not in seen:
            out.append(p)
            seen.add(p)
    return out

# These labels are NOT cuisines; they are marketing/platform tags
STOP_TAGS = {
    "lokale geheimtipps",
    "angebote",
    "traditionell",
    "geschäfte",   # sometimes appears; not cuisine
}

def filter_stop_tags(tags: list) -> list:
    if not isinstance(tags, list):
        return []
    return [t for t in tags if t not in STOP_TAGS]

df["description_clean"] = df["description"].apply(normalize_text)
df["cuisine_tags_raw"] = df["description"].apply(split_tags)
df["cuisine_tags"] = df["cuisine_tags_raw"].apply(filter_stop_tags)

df["num_cuisines_raw"] = df["cuisine_tags_raw"].apply(len)
df["num_cuisines"] = df["cuisine_tags"].apply(len)

# --- Better rules: "snacks" is treated as generic (not a main cuisine)
TAXONOMY_RULES = [
    ("grocery_retail", [
        r"\blebensmittel\b", r"\bsupermarkt\b", r"\bkiosk\b", r"\bshop\b",
        r"\bmetzgerei\b", r"\bb(a|ä)ckerei\b"
    ]),
    ("beverages_snacks", [
        r"\bgetr(a|ä)nke\b", r"\bkaffee\b", r"\bcoffee\b", r"\btee\b",
        r"\bbubble tea\b", r"\bsmoothies?\b"
    ]),
    ("desserts", [
        r"\bnachspeisen\b", r"\bdessert(s)?\b", r"\bdonuts?\b", r"\beis\b"
    ]),
    ("dietary_lifestyle", [
        r"\bvegan\b", r"\bvegetarisch\b", r"\bhalal\b"
    ]),
    ("asian", [
        r"\basiatisch\b", r"\bthail(a|ä)ndisch\b", r"\bchinesisch\b",
        r"\bvietnamesisch\b", r"\bjapanisch\b", r"\bsushi\b",
        r"\bkorean\b", r"\bindisch\b", r"\bmomos?\b", r"\bdumplings\b", r"\bpho\b"
    ]),
    ("middle_eastern", [
        r"\bkebab\b", r"\bd(o|ö)ner\b", r"\bfalafel\b",
        r"\barabisch\b", r"\blibanesisch\b", r"\bt(u|ü)rkisch\b", r"\bpersisch\b", r"\bburek\b"
    ]),
    ("american", [
        r"\bamerikanisch\b", r"\bburger\b", r"\bsandwich(es)?\b", r"\bwraps?\b",
        r"\bh(u|ü)hnchen\b", r"\bfried chicken\b"
    ]),
    ("european", [
        r"\bitalienisch\b", r"\bitalienische pizza\b", r"\bpizza\b", r"\bpasta\b",
        r"\bgriechisch\b", r"\bbalkank(u|ü)che\b"
    ]),
    ("latin", [r"\bmexikanisch\b", r"\btacos?\b"]),
    ("african", [r"\bafrikanisch\b", r"\bafro\b"]),
]

GENERIC_TAGS = {"snacks"}  # does not define cuisine group by itself

def tag_to_groups(tag: str) -> list:
    if not tag or tag in GENERIC_TAGS:
        return []
    groups = []
    for group, patterns in TAXONOMY_RULES:
        for pat in patterns:
            if re.search(pat, tag):
                groups.append(group)
                break
    return sorted(set(groups))

def tags_to_groups(tags: list) -> list:
    if not isinstance(tags, list) or len(tags) == 0:
        return []
    out = []
    for t in tags:
        out.extend(tag_to_groups(t))
    return sorted(set(out))

df["cuisine_groups"] = df["cuisine_tags"].apply(tags_to_groups)
df["num_cuisine_groups"] = df["cuisine_groups"].apply(len)

# Priority: pick a "real cuisine" first, then business types
PRIMARY_PRIORITY = [
    "asian", "middle_eastern", "american", "european", "latin", "african",
    "grocery_retail", "beverages_snacks", "desserts", "dietary_lifestyle"
]

def choose_primary_group(groups: list) -> str:
    if not isinstance(groups, list) or len(groups) == 0:
        return "unknown_other"
    for g in PRIMARY_PRIORITY:
        if g in groups:
            return g
    return "unknown_other"

df["primary_cuisine_group"] = df["cuisine_groups"].apply(choose_primary_group)

# Dietary flags (kept even if not primary)
df["has_vegan"] = df["description_clean"].str.contains(r"\bvegan\b").astype(int)
df["has_vegetarian"] = df["description_clean"].str.contains(r"\bvegetarisch\b").astype(int)
df["has_halal"] = df["description_clean"].str.contains(r"\bhalal\b").astype(int)

# Business type flags
df["is_grocery_or_shop"] = (df["primary_cuisine_group"] == "grocery_retail").astype(int)
df["is_beverage_snack"] = (df["primary_cuisine_group"] == "beverages_snacks").astype(int)
df["is_dessert"] = (df["primary_cuisine_group"] == "desserts").astype(int)

In [24]:
# ==========================================================
# 6) SHOW RESULTS (CLEAN + ORDERED OUTPUT)
# ==========================================================

# Make list-columns readable (comma-separated) for printing/export
pretty = df.copy()

pretty["cuisine_groups"] = pretty["cuisine_groups"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# Optional: make tag columns readable if you want them later
pretty["cuisine_tags_raw"] = pretty["cuisine_tags_raw"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)
pretty["cuisine_tags"] = pretty["cuisine_tags"].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else ""
)

# ORDERED columns for the report (simple + clean)
cols_show = [
    "name",
    "rating",
    "minimum_clean",
    "primary_cuisine_group",
    "cuisine_groups",
]

# Sort rows for readability (by cuisine group, then rating desc)
pretty_sorted = pretty.sort_values(
    by=["primary_cuisine_group", "rating"],
    ascending=[True, False]
)

# Print sample
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", 40)

print("\n--- CLEAN REPORT VIEW (sample 20 rows) ---")
print(
    pretty_sorted
        .loc[:, cols_show]
        .head(20)
        .to_string(index=False)
)

# ==========================================================
# Distribution of cuisine groups
# ==========================================================
print("\n--- primary_cuisine_group distribution ---")
print(pretty["primary_cuisine_group"].value_counts(dropna=False))

# ==========================================================
# Summary by cuisine group (NO dietary flags)
# ==========================================================
summary = (
    pretty.groupby("primary_cuisine_group", dropna=False)
          .agg(
              n=("name", "count"),
              avg_rating=("rating", "mean"),
              avg_minimum=("minimum_clean", "mean"),
              median_minimum=("minimum_clean", "median"),
          )
          .sort_values("n", ascending=False)
)

print("\n--- summary by primary_cuisine_group ---")
print(summary.to_string())

# ==========================================================
# 7) SAVE FILES
# ==========================================================
df.to_csv("zurich_enriched_taxonomy.csv", index=False)
pretty_sorted.loc[:, cols_show].to_csv("zurich_report_view.csv", index=False)

print("\nSaved file: zurich_enriched_taxonomy.csv (full enriched dataset)")
print("Saved file: zurich_report_view.csv (clean ordered view for Excel/report)")



--- CLEAN REPORT VIEW (sample 20 rows) ---
                                                name  rating  minimum_clean primary_cuisine_group     cuisine_groups
                              Happen Afro Restaurant     5.0           20.0               african            african
                                    Restaurant Mesob     4.9           20.0               african            african
                                 S-Family Restaurant     4.7           20.0               african            african
                                  Habesha Restaurant     4.5           20.0               african            african
                           Cindys Bistro - Afro Deli     4.3           20.0               african            african
                                     Taste of Africa     4.3           20.0               african            african
                                     Cafe Bar Balkan     5.0           35.0              american           american
                    